In [2]:
import pandas as pd
import numpy as np
import requests 
import json
requests.get('https://api.github.com')


<Response [200]>

Importation des librairies et test de la libraire requests

In [22]:
account = pd.read_csv("account_groundTruth[92].csv")
account = account.iloc[:,[1,2]] #used because otherwise the account number are there twice 
# 0              0                       brotherlogic     bot        
#instead of
#0                       brotherlogic     bot
#print(account.to_string())
account.describe

<bound method NDFrame.describe of               login type_GT
0      brotherlogic     bot
1         coveralls     bot
2          renovate     bot
3          leocwlam     bot
4         fabric8cd     bot
...             ...     ...
1487       raphaelm   human
1488     TroyAlford   human
1489          daffl   human
1490  bryanyang0528   human
1491    ConnorRigby   human

[1492 rows x 2 columns]>

Utilisation de panda pour récupérer la base de données contenant les bots et compte humains.

In [23]:
values = account.values
checker = dict(values)
checker["brotherlogic"]

'bot'

Création d'un dictionnaire pour que lorsque la fonction de recherche pour les bots sera établie, si celle ci a bien détecté un bot ou non.

In [17]:
"""
def check(login,boolBot):
    if (boolBot):
        return (checker[login] == "bot")
    else:
        return (checker[login] == "human")
    
print(check("brotherlogic",True))
print(check("coveralls",False))
print(check("ConnorRigby",False))
print(check("ConnorRigby",True))
"""
def checkbot(login):
    return (checker[login] == "bot")

def checkhuman(login):
    return (checker[login] == "human")



Fonctions servant juste à vérifier correctement dans le dictionnaire

In [24]:
b_set = account.head(25) #bot set of size 25
h_set = account.tail(25) #human set of size 25
m_set = pd.concat([b_set,h_set])                 #mixed set of size 50
m_set = m_set.reset_index(drop = True)           #maybe not needed but "cleans" the dataframe
                                                 # so it goes from 1 to 50 instead of 1 to 25 then 1467 to 1491
print(m_set.to_string())
print(b_set.to_string())
print(h_set.to_string())

                               login type_GT
0                       brotherlogic     bot
1                          coveralls     bot
2                           renovate     bot
3                           leocwlam     bot
4                          fabric8cd     bot
5                       codecov[bot]     bot
6                         codecov-io     bot
7                     ant-design-bot     bot
8                facebook-github-bot     bot
9                           snyk-bot     bot
10             jnpr-community-netdev     bot
11           dependabot-preview[bot]     bot
12                           ezrobot     bot
13                thelinuxfoundation     bot
14                      Hibernate-CI     bot
15                         cf-gitbot     bot
16                        stale[bot]     bot
17              scrutinizer-notifier     bot
18                         googlebot     bot
19                              bors     bot
20                      k8s-ci-robot     bot
21        

Création d'une dataframe plus petite pour commencer

In [33]:
#r = requests.get('https://api.github.com/events')
for i in range( 1,4):
    payload = {'per_page' : '100' ,'page' : '{}'.format (i)}
    r = requests.get("https://api.github.com/users/brotherlogic/events",params=payload)
    activity = r.json()
    with open ("./test/test{}.json".format(i),"w") as outfile:
        json.dump(activity,outfile, indent = 4)
        
outfile.close()




In [44]:
payload = {'per_page' : '100' ,'page' : '1'}
r = requests.get("https://api.github.com/users/brotherlogic/events",params=payload)
activity = r.json()
with open ("./new.json","w") as outfile:
    json.dump(activity,outfile, indent = 4)
outfile.close()

In [58]:
#test1 = json.load("./events.json")
#test2 = json.load("./activities/TroyAlford2.json")
test1 = json.load(open("./activities/TroyAlford1.json"))
test2 = json.load(open("./activities/TroyAlford2.json"))
print(test1 == test2)


True


In [64]:
payload1 = {'per_page' : '100' ,'page' : '1'}
payload2 = {'per_page' : '100' ,'page' : '2'}
payload3 = {'per_page' : '100' ,'page' : '3'}

r1 = requests.get("https://api.github.com/users/TroyAlford/events",params=payload1)
r2 = requests.get("https://api.github.com/users/TroyAlford/events",params=payload2)
r3 = requests.get("https://api.github.com/users/TroyAlford/events",params=payload3)


activity1 = r1.json()
activity2 = r2.json()
activity3 = r3.json()
with open("./test1.json","w") as outfile:
                json.dump(activity1,outfile,indent = 4)
with open("./test2.json","w") as outfile:
                json.dump(activity2,outfile,indent = 4)
print("r1 r2", "{}".format(activity1==activity2))
print("r2 r3", "{}".format(activity2==activity3))
print("r1 r3", "{}".format(activity1==activity3))


r1 r2 False
r2 r3 True
r1 r3 False


In [67]:
def getactivities(list_login):
    for login in list_login:
        for i in range (1,4):
            payload = {'per_page' : '100' ,'page' : '{}'.format(i)}
            r = requests.get("https://api.github.com/users/{name}/events".format(name=login),params=payload)
            activity = r.json()
            with open("./activities/{name}{number}.json".format(name = login,number = i),"w") as outfile:
                json.dump(activity,outfile,indent = 4)
            outfile.close()

Création d'une fonction pour récupérer les activités des comptes.
Vérifier pourquoi certains fichiers json sont juste sous la forme root: "[]"
Vérifier aussi pourquoi les fichiers ne sont plus "reconnu" comme json (regarder fichier test.json et event.json)
Problème réglés (changer le r.text en r.json() )
Ne marche plus après être passé sur vscode

In [69]:
sample = account.head(5)
login_sample = sample.login.values.tolist()
print(login_sample)

getactivities(login_sample)

['brotherlogic', 'coveralls', 'renovate', 'leocwlam', 'fabric8cd']


In [70]:
sample1 = account.tail(5)
login_sample1 = sample1.login.values.tolist()
print(login_sample1)

getactivities(login_sample1)

['raphaelm', 'TroyAlford', 'daffl', 'bryanyang0528', 'ConnorRigby']


Utilisation de la fonction et récupération une première fois des activités des comptes

In [10]:
df = pd.read_json("./activities/TroyAlford1.json")
df.describe
lst = df.keys()
print(lst)
df.get('type')

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')


0    PushEvent
1    PushEvent
2    PushEvent
Name: type, dtype: object

In [38]:
combined = []
for i in range (1,4):
    tmp = pd.read_json("./activities/TroyAlford{index}.json".format(index = i))
    combined.append(tmp)
# res = pd.concat(combined)
# res = res.reset_index(drop = True)
res= (pd.concat(combined)).reset_index(drop = True)
res.head()
# display(res.iloc[0])
# display(res.iloc[1])


,id,type,actor,repo,payload,public,created_at,org
0,24905564307,PushEvent,"{'id': 12262780, 'login': 'TroyAlford', 'displ...","{'id': 393739387, 'name': 'NerdrageTools/7d2d-...","{'push_id': 11488521095, 'size': 1, 'distinct_...",True,2022-10-29 07:32:21+00:00,"{'id': 33378902, 'login': 'NerdrageTools', 'gr..."
1,24782205542,PushEvent,"{'id': 12262780, 'login': 'TroyAlford', 'displ...","{'id': 393739387, 'name': 'NerdrageTools/7d2d-...","{'push_id': 11426088539, 'size': 8, 'distinct_...",True,2022-10-24 06:37:55+00:00,"{'id': 33378902, 'login': 'NerdrageTools', 'gr..."
2,24491681996,PushEvent,"{'id': 12262780, 'login': 'TroyAlford', 'displ...","{'id': 393739387, 'name': 'NerdrageTools/7d2d-...","{'push_id': 11275036462, 'size': 1, 'distinct_...",True,2022-10-10 03:10:37+00:00,"{'id': 33378902, 'login': 'NerdrageTools', 'gr..."


Global idea on how to get a good dataframe based on the json we got

In [37]:
def getdataframe(login):
    combined=[]
    for i in range (1,4):
        tmp = pd.read_json("./activities/{}{}.json".format(login,i))
        combined.append(tmp)
    res= (pd.concat(combined)).reset_index(drop = True)
    return res

getdataframe("coveralls")

,id,type,actor,repo,payload,public,created_at,org
0,24917066563,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 557388363, 'name': 'PW-mates/cinemo-bac...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-30 14:27:32+00:00,"{'id': 116636290, 'login': 'PW-mates', 'gravat..."
1,24917066422,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 7514061, 'name': 'MushroomObserver/mush...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-30 14:27:31+00:00,"{'id': 3212163, 'login': 'MushroomObserver', '..."
2,24917018562,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 91629816, 'name': 'freqtrade/freqtrade'...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-30 14:21:14+00:00,"{'id': 37536846, 'login': 'freqtrade', 'gravat..."
3,24917004924,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 489777536, 'name': 'aas-core-works/aas-...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-30 14:19:29+00:00,"{'id': 81610708, 'login': 'aas-core-works', 'g..."
4,24916976229,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 3101686, 'name': 'thumbor/remotecv', 'u...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-30 14:15:43+00:00,"{'id': 6918711, 'login': 'thumbor', 'gravatar_..."
...,...,...,...,...,...,...,...,...
295,24901969823,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 109200851, 'name': 'microsoft/azuredata...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-28 22:07:48+00:00,"{'id': 6154722, 'login': 'microsoft', 'gravata..."
296,24901952702,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 436729130, 'name': 'openconfig/featurep...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-28 22:06:26+00:00,"{'id': 8780263, 'login': 'openconfig', 'gravat..."
297,24901798205,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 162741822, 'name': 'decentraland/builde...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-28 21:53:44+00:00,"{'id': 12685795, 'login': 'decentraland', 'gra..."
298,24901788984,IssueCommentEvent,"{'id': 2354108, 'login': 'coveralls', 'display...","{'id': 360238600, 'name': 'medplum/medplum', '...","{'action': 'created', 'issue': {'url': 'https:...",True,2022-10-28 21:52:59+00:00,"{'id': 75462971, 'login': 'medplum', 'gravatar..."


In [ ]:
df = pd.read_json("./activities/coveralls3.json")

df.describe
lst = df.keys()
#print(lst)
#print(df['type'].values)
dict = {}
for val in df['type'].values:
    if val not in dict:
        dict[val]= 1
    else:
        dict[val] = dict[val] +1
print(dict)
#print(lst)
# lst1 = df['id'].values
# print(lst1)
# df.get('type')
# df.get('repo')


{'IssueCommentEvent': 100}


Global idea on how to get how many time the same action is done by a certain account

In [ ]:
def count_type(login):
    df = getdataframe(login)
    dict = {}
    for val in df['type'].values:
        if val not in dict:
            dict[val]= 1
        else:
            dict[val] = dict[val] +1
    return dict